# Questions
1. 지역에 따라서 선호하는 게임 장르가 다를까
2. 연도별 게임의 트렌드가 있을까 
3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

- <span style="font-size:18px; font-style:italic; font-weight:bold">Goal: 다음 분기에 어떤 게임을 설계해야 할까</span>
<br>

# Import and Load

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('vgames2.csv', index_col=0)

In [3]:
# Metacritic Scores 
# https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings
scores = pd.read_csv('vgsales_score.csv')

# Quick view

## Main Dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


### Metadata

- `Name` : 게임의 이름입니다.
- `Platform` : 게임이 지원되는 플랫폼의 이름입니다. (i.e. PC, PS4, etc.)
- `Year` : 게임이 출시된 연도입니다.
- `Genre` : 게임의 장르입니다.
- `Publisher` : 게임을 제작한 회사입니다.
- `NA_Sales` : 북미지역에서의 출고량입니다. (in millions)
- `EU_Sales` : 유럽지역에서의 출고량입니다. (in millions)
- `JP_Sales` : 일본지역에서의 출고량입니다. (in millions)
- `Other_Sales` : 기타지역에서의 출고량입니다.(in millions)

In [5]:
display(df.head())
df.shape

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


(16598, 9)

In [6]:
df.isna().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

## Score Dataset

In [7]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [8]:
scores.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
scores.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

---

# Preprocessing

## 데이터 타입 수정, 중복값 제거

In [10]:
df_clean = df.drop_duplicates()

In [11]:
# change Sales dtypes, object to float
def sale_convert(data, cols):
    for col in cols:
        temp = data[(data[col].str[-1]=='K') | (data[col].str[-1]=='M')]
        data.loc[temp.index, col] = temp[col].map(lambda x: float(x[:-1])*0.001 if x[-1]=='K' else float(x[:-1]))
        data[col] = data[col].astype(float)
    return data

sales = [col for col in df.columns if col.endswith('Sales')]
df_clean = sale_convert(df.copy(), sales)

In [12]:
df_clean['Total_Sales'] = df_clean[sales].sum(axis=1)

In [13]:
df_clean[df_clean.duplicated(keep=False)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
8056,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02
9185,Wii de Asobu: Metroid Prime,Wii,NaN,Shooter,Nintendo,0.0,0.0,0.02,0.0,0.02


In [14]:
df_clean = df_clean.drop_duplicates()

In [15]:
df_clean[df_clean.duplicated(subset=['Name', 'Platform'], keep=False)].sort_values(['Name', 'Platform'])

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
7347,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,2.11,0.23,0.00,0.22,2.56
8718,Madden NFL 13,PS3,2012.0,Sports,Electronic Arts,0.00,0.01,0.00,0.00,0.01
2674,Need for Speed: Most Wanted,PC,2005.0,Racing,Electronic Arts,0.02,0.23,0.00,0.05,0.30
13809,Need for Speed: Most Wanted,PC,2012.0,Racing,Electronic Arts,0.00,0.06,0.00,0.02,0.08
1319,Need for Speed: Most Wanted,X360,2012.0,Racing,Electronic Arts,0.62,0.80,0.01,0.15,1.58
12530,Need for Speed: Most Wanted,X360,2005.0,Racing,Electronic Arts,1.00,0.17,0.02,0.10,1.29
8652,Sonic the Hedgehog,PS3,NaN,Platform,NaN,0.00,0.48,0.00,0.00,0.48
15706,Sonic the Hedgehog,PS3,2006.0,Platform,Sega,0.41,0.07,0.04,0.66,1.18


Madden NFL 13랑 Sonic the Hedgehog는 같은 플랫폼인데도 따로 있으니 합쳐준다

In [16]:
temp_Madden = df_clean.query("Name=='Madden NFL 13' and Platform=='PS3'")[sales+['Total_Sales']].sum()
temp_Sonic = df_clean.query("Name=='Sonic the Hedgehog' and Platform=='PS3'")[sales+['Total_Sales']].sum()

In [17]:
df_clean = df_clean.drop([8718, 15706])

In [18]:
df_clean.loc[7347, sales+['Total_Sales']] = temp_Madden
df_clean.loc[8652, sales+['Total_Sales']] = temp_Sonic

In [19]:
df_clean[df_clean.duplicated(subset=['Name', 'Platform'], keep=False)].sort_values(['Name', 'Platform'])

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
2674,Need for Speed: Most Wanted,PC,2005.0,Racing,Electronic Arts,0.02,0.23,0.00,0.05,0.30
13809,Need for Speed: Most Wanted,PC,2012.0,Racing,Electronic Arts,0.00,0.06,0.00,0.02,0.08
1319,Need for Speed: Most Wanted,X360,2012.0,Racing,Electronic Arts,0.62,0.80,0.01,0.15,1.58
12530,Need for Speed: Most Wanted,X360,2005.0,Racing,Electronic Arts,1.00,0.17,0.02,0.10,1.29


In [20]:
# score에 있는 문자열 데이터 수정
scores['User_Score'] = scores['User_Score'].replace({'tbd': np.nan}).astype(float)

## 연도 결측치, 오류 처리

연도가 두 자리인 경우 4자리로 맞춰준다

In [21]:
df_clean.loc[df_clean['Year']<1900, 'Year'] = (df_clean.loc[df_clean['Year']<1900, 'Year']
                                               .map(lambda x: x+2000 if x <= 22 else x+1900))

전체 출고량을 기준으로 높은 출고량의 경우에는 연도를 채워주고 그렇지 않은 게임은 삭제한다.

In [22]:
df_clean = df_clean.merge(scores, how='left')

In [23]:
df_clean = df_clean.drop(['Global_Sales', 'Year_of_Release', 'Critic_Count',
                          'User_Count', 'Developer', 'Rating'], axis=1)

In [24]:
df_clean['Total_Sales'].median(), df_clean['Total_Sales'].mean()

(0.17, 0.5372600180777343)

총 판매량의 중앙값이 0.17,<br>
평균은 0.537 <br>

데이터가 판매량이 낮은 쪽으로 편향되어있어 중앙값이 더 정확하지만 적게 많이 팔린 게임보다는 많이 팔린 게임이 트랜드를 더 잘 반영한다고 가정함

In [25]:
# cond = ((df_clean['Year'].isna()) & ((df_clean['Total_Sales']>0.17) | ((df_clean['Critic_Score']>=78) &
#         (df_clean['User_Score']>8.3))))
cond = (df_clean['Year'].isna()) & (df_clean['Total_Sales']>0.537)

~~중복된 이름중에 연도 없는건 있는걸로 대체~~ 같은 이름은 다른 게임도 있어서 안됨

연도 아에 없는거 채워야됨

전체 다 하면 오래걸리니까 ~~전체 기간동안 연도별 중앙값의~~ 평균보다 높은 값만 직접 찾아서 써줌
('Total_Sales > 0.537) <br>
같은 게임은 다른 플랫폼에서 출시되서 연도 다른건 앞선 연도 사용

In [26]:
missing_year_names = set(df_clean[cond]['Name'])

In [27]:
missing_year = [2008, 1978, 1977, 2006, 2010, 1977, 2004, 1997, 2003, 2001,
                2002, 2008, 2011, 2008, 1999, 2002, 2005, 2006, 2010, 2001,
                2003, 2002, 2006, 2002, 2002, 2002, 2008, 2008, 2007, 2008,
                2006, 1978, 2002, 2011, 2005, 2007, 2011, 1998, 2005, 2003,
                2011, 2005]

In [28]:
years_fill = dict(zip(sorted(missing_year_names), missing_year))

In [29]:
df_clean['Year'] = df_clean['Year'].fillna(df_clean['Name'].map(years_fill))

적은 출고량의 데이터 삭제

In [30]:
df_clean = df_clean.dropna(subset='Year')

In [31]:
df_clean['Year'] = df_clean['Year'].astype(int)

2016년까지 출시된 게임 목록인데 그 이후에 출시된 게임이 있다.
출시 일자 확인 후 수정해준다.

In [32]:
df_clean[df_clean['Year']>2016]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales,Critic_Score,User_Score
5310,Brothers Conflict: Precious Baby,PSV,2017,Action,Idea Factory,0.00,0.0,0.01,0.00,0.01,NaN,NaN
6906,Imagine: Makeup Artist,DS,2020,Simulation,Ubisoft,0.27,0.0,0.00,0.02,0.29,NaN,NaN
10105,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017,Role-Playing,Sega,0.00,0.0,0.03,0.00,0.03,NaN,NaN
15231,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017,Role-Playing,Sega,0.00,0.0,0.01,0.00,0.01,NaN,NaN


In [33]:
df_clean.loc[df_clean['Name']=='Brothers Conflict: Precious Baby', 'Year'] = 2016
df_clean.loc[df_clean['Name']=='Imagine: Makeup Artist', 'Year'] = 2009
df_clean.loc[df_clean['Name']=='Phantasy Star Online 2 Episode 4: Deluxe Package', 'Year'] = 2016

## 장르 결측치 처리

장르는 빠르게 추가하기 위해 [외부 데이터셋](https://www.kaggle.com/datasets/gregorut/videogamesales)에 있는 장르 가져왔음 

In [34]:
vgsales = pd.read_csv('vgsales.csv')

In [35]:
genres = vgsales[['Name', 'Genre']].drop_duplicates()
genres = {k: v for (k,v) in genres.values}

In [36]:
df_clean['Genre'] = df_clean['Genre'].fillna(df_clean['Name'].map(genres))

In [37]:
df_clean.isna().sum()

Name                0
Platform            0
Year                0
Genre               0
Publisher          39
NA_Sales            0
EU_Sales            0
JP_Sales            0
Other_Sales         0
Total_Sales         0
Critic_Score    11022
User_Score      11657
dtype: int64

## 퍼블리셔 결측치 처리

In [38]:
cond = (df_clean['Publisher'].isna()) & (df_clean['Total_Sales']>0.537)

In [39]:
missing_publisher_names = set(df_clean[cond]['Name'])

In [40]:
missing_publisher = ['Sony Computer Entertainment', 'Majesco', 'Majesco', 'Sega',
                     'Majesco', 'EA Sports', 'THQ']

In [41]:
publisher_fill = dict(zip(sorted(missing_publisher_names), missing_year))

In [42]:
df_clean['Publisher'] = df_clean['Publisher'].fillna(df_clean['Name'].map(publisher_fill))

적은 판매량의 데이터 삭제

In [43]:
df_clean = df_clean.dropna(subset='Publisher')

In [44]:
df_clean.shape

(16350, 12)

In [45]:
df_clean.isna().sum()

Name                0
Platform            0
Year                0
Genre               0
Publisher           0
NA_Sales            0
EU_Sales            0
JP_Sales            0
Other_Sales         0
Total_Sales         0
Critic_Score    10991
User_Score      11627
dtype: int64

In [46]:
df_clean.to_csv('vgames2_clean.csv', index=False)